In [1]:
from dotenv import load_dotenv
import os
from edapi import EdAPI
from edapi.models.user import User
import pandas as pd
load_dotenv()

True

In [2]:
ED_API_TOKEN = os.getenv("ED_API_TOKEN")
ed_client = EdAPI(ED_API_TOKEN)

user:User = ed_client.get_user_info()

print(f"Hello {user.name}!")


Hello Caleb Ooi!


Get unit thats active

In [3]:
active_course = user.get_active_courses()
active_course_df = pd.DataFrame([course.to_dict() for course in active_course])
target_course = "Sandbox FIT5032"
course_id = 14448

Get threads


In [4]:

thread_responses = ed_client.list_threads(course_id=course_id,limit=100)
print(thread_responses[0])
thread_responses_df = pd.DataFrame([thread.model_dump() for thread in thread_responses])
thread_responses_df.shape


id=1816687 user_id=114866 course_id=14448 type='post' title='🌟 Frequently Asked Questions' document='Unit Logistics\n\nDue to some circumstances, I cannot meet the submission deadline; what should I do?\n\nApply for special consideration. You can find more details under the Assignments control panel.\n\nWhere can I find my tutor\'s emails?\n\nYou can find them on Moodle under the Unit Information section.\n\nCan I use libraries or frameworks not delivered in this unit in my assignments?\n\nNo. \n\nWhy are the tutors not replying to me?\n\nEmailing staff after hours\nWhile we try to respond to enquiries as soon as possible, please do not expect immediate replies to emails, especially outside of regular working hours (or non-teaching periods e.g. mid-semester break). It may take up to 2 working days to receive an answer to an email or Ed post.\n\nIf you send emails to teaching staff on Friday, be prepared to wait until Monday or Tuesday for a reply.\n\nTechnical issues\n\nHow do I ask te

(100, 14)

Cleaning up the data

In [5]:
#combine the title, document and category 
thread_responses_df['combine_content'] = thread_responses_df['title'] + " " + thread_responses_df['document'] + " " + thread_responses_df['category']

# thread_responses_df
#remove empty string e.g \n \t
thread_responses_df['chunk_text'] = thread_responses_df['combine_content'].str.strip()

#pine df needs id, values, metadata
pine_df = thread_responses_df[['id','chunk_text','category']].copy()
pine_df['id'] = pine_df['id'].astype(str)
pine_df

,id,chunk_text,category
0,1816687,🌟 Frequently Asked Questions Unit Logistics\n\...,General
1,2085356,Supplementary How do we apply for supplementar...,General
2,2084004,Analysis of final grade Can we get a breakdown...,General
3,2075399,Supplementary Will there be any supplementary ...,General
4,2071709,"Unmarked Labs Hi there, I was wondering when a...",Labs
...,...,...,...
95,1977903,"Snackbar on save of Event Hi,\n\nDo we need to...",Assignments
96,1977398,Google maps back button problem with Intent da...,Assignments
97,1976409,"Grade Distribution Hi, \n\nIt says if we compl...",Assignments
98,1975705,Room Database error Assignments,Assignments


insert into vector db

In [6]:
from pinecone import Pinecone
PINE_CONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINE_CONE_API_KEY)

c:\Users\Caleb\Documents\FYP\Ed-summariser-server\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
index_name = target_course.lower().replace(" ", "-") + "-thread-embeddings"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model": "llama-text-embed-v2",
            "field_map": {"text": "chunk_text"}
        }
    )

In [8]:
index = pc.Index(index_name)
for i in range(0, pine_df.shape[0], 50):
    print(f"Upserting records {i} to {i+50}")
    index.upsert_records(namespace="test", records=pine_df.to_dict(orient='records')[i:i+50])

Upserting records 0 to 50
Upserting records 50 to 100


k mean clustering

In [9]:
from sklearn.cluster import KMeans
from numpy import random

In [10]:
ids = list(index.list(namespace="test"))
embedding = index.fetch(ids[0], namespace="test")
# {id, values}
embeddings_dict = embedding.vectors
embeddings = [{"id": vector.id, "embedding": vector.values, "metadata": vector.metadata } for vector in embeddings_dict.values()]
embeddings_df = pd.DataFrame(embeddings)


In [11]:
embeddings_array = embeddings_df['embedding'].to_list()

In [14]:
kmeans = KMeans(n_clusters=8, random_state=random.randint(0,100000)).fit(embeddings_array)
clusters= kmeans.cluster_centers_
search = clusters[0].tolist()


In [16]:
results = index.query(vector=search, top_k=3, namespace="test", rerank={
    "model": "bge-reranker-v2-m3",
    "top_n": 10,
    "rank_fields": ["chunk_text"]
})
matched_ids = [result.id for result in results.matches]
## get the content 
related_questions = index.fetch(matched_ids, namespace="test")

In [18]:
text = [thread.metadata for thread in related_questions.vectors.values()]
text_combine = "question :".join([f"{t["chunk_text"]}" for t in text])


In [21]:
text_combine

'Error that I cannot solve I keep getting this error when trying to get the categoryID of the event category  Assignmentsquestion :Error saying cannot be instantiated Hello,\n\neverytime I try to create an instance of a class ( lets just say Event class for example)\n\ni keep getting an error saying Event is abstract, cannot be instantiated and it doesnt make sense bc my class is not abstract  Assignmentsquestion :ERR_NAME_NOT_RESOLVED Please help me fix this error, it was working all well, and now it has suddenly displayed this error upon click of the event card for the Wiki page:\n\n Assignments'

Send the cluster related document to llm to create faq

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=(
        ""),
)

In [ ]:

completion = client.chat.completions.create(
    model="google/gemini-2.0-flash-lite-preview-02-05:free",
  messages=[
    {
      "role": "system",
      "content": "You are an assistant for a teaching of a large unit at a university. \
You are given a set of questions that are related, generate a concise questions that caputre all the important details of these questions so that the teaching team can answer them. \
If the questions are not related, please say that they are not related. \
The provided text contains questions and the category of the qestions which can be General, Labs, Assignments. Please also group the questions by their category. \""
    },
      {
        "role": "system",
        "content": "Follow the return format below: \
          {\
          'questions': []"
          },
    {
        "role": "user",
        "content": text_combine
    }
   
  ]
)
print(completion)
print(completion.choices[0].message.content)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (156540988.py, line 11)